# SVM 하이퍼파라미터 최적화 - 랜덤 서치
랜덤 서치는 하이퍼파라미터의 값들을 랜덤으로 선택하여 모델을 학습시키고, 그 결과를 바탕으로 하이퍼파라미터의 최적 조합을 찾는 방법<br>
랜덤 서치를 사용하면 하이퍼파라미터의 모든 조합을 시도해보는 것보다 효율적으로 하이퍼파라미터를 탐색할 수 있다. 하지만 랜덤 서치 역시 하이퍼파라미터의 범위를 어떻게 설정하느냐에 따라 최적의 조합을 찾는데 실패할 수도 있다.<br>

- 랜덤 서치 구현
1. SVM 모델 정의, 모델 학습 함수 작성
2. 랜덤으로 선택할 하이퍼파라미터의 범위 설정
3. 2번을 바탕으로 랜덤하게 하이퍼파라미터를 선택해 SVM 모델 학습
4. 검증 데이터를 바탕으로 모델의 성능 평가
5. 위 과정을 여러번 반복하여 최적의 하이퍼파라미터 조합을 찾는다.

### 랜덤 서치 실습

In [1]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from scipy.stats import loguniform
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score

In [2]:
# 데이터 로드
digits = load_digits()

# 독립 변수, 종속 변수 데이터 지정
x = digits.data
y = digits.target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
# 손글씨 숫자 이미지 데이터 로드, train_test_split() 함수를 사용해 train data와 test data로 분리하는 코드

### 파이프라인 구성

In [3]:
svm_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC())
])

### 탐색할 하이퍼파라미터 공간

In [4]:
params = {
    "C" : loguniform(1e-4, 100),
    "kernel" : ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma' : ['scale', 'auto'] + list(loguniform(1e-4, 10).rvs(10)),
    'degree' : range(1,6),
    'coef0': loguniform(1e-4, 10).rvs(10)
}

- C : SVM 모델의 규제 파라미터. 1e-4부터 100까지 log scale 범위에서 탐색
- kernel : SVM 모델에서 사용할 커널 함수 설정. linear, poly, rbf, sigmoi 중 하나 사용
- gamma : SVM 모델에서 사용할 커널 함수의 폭에 해당하는 파라미터. scale, auto를 포함한 1e-4부터 10까지 log scale 범위에서 탐색한다.
- degree : poly 커널 함수에서 사용할 차수에 해당하는 파라미터. 1~5까지의 값 중 하나를 사용
- coef0 : poly, sigmoid 커널 함수에서 사용할 상수항에 해당하는 파라미터이다. 1e-부터 10까지 log scale 범위에서 탐색

#### rvs() 함수는 rv_frozen 객체에서 random variates(랜덤 샘플)을 생성한다.

### 랜덤 서치 실행

In [6]:
# SVM 모델 생성
svm_model = SVC()

# 랜덤 서치 객체 생성
random_search = RandomizedSearchCV(svm_model, params, n_iter=100, cv=5, verbose=2, n_jobs=-1)

# 랜덤 서치 수행
random_search.fit(x_train, y_train)

# 최적 하이퍼파라미터 출력
print('Best hyperparameters : ', random_search.best_params_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best hyperparameters :  {'C': 39.44419812306922, 'coef0': 0.00016601468781762967, 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf'}


In [7]:
# 최적 모델 저장
best_model = random_search.best_estimator_

# 테스트 데이터에 대한 예측 수행
y_pred = best_model.predict(x_test)

# 정확도 출력
print('Test accuracy : ', accuracy_score(y_test, y_pred))

Test accuracy :  0.9861111111111112
